In [ ]:
# list_docreq=["gov.tr","bel.tr","pol.tr","edu.tr","k12.tr","av.tr","dr.tr","gov.ct.tr","tsk.tr"]
# list_nodoc=["com.tr","net.tr","org.tr","biz.tr","info.tr","tv.tr","gen.tr","web.tr","name.tr","bbs.tr"]
# list_other=["istanbul","ist"]
domain="bbs.tr"

In [ ]:
!wget "https://github.com/OWASP/Amass/releases/download/v3.21.2/amass_linux_amd64.zip"
!unzip "amass_linux_amd64.zip" -d "/content/tools"
!rm "amass_linux_amd64.zip"

!wget "https://github.com/projectdiscovery/subfinder/releases/download/v2.5.5/subfinder_2.5.5_linux_amd64.zip"
!unzip "subfinder_2.5.5_linux_amd64.zip" -d "/content/tools"
!rm "subfinder_2.5.5_linux_amd64.zip"

!wget "https://github.com/projectdiscovery/httpx/releases/download/v1.2.5/httpx_1.2.5_linux_amd64.zip"
!unzip "httpx_1.2.5_linux_amd64.zip" -d "/content/tools"
!rm "httpx_1.2.5_linux_amd64.zip"

`1. Domain Enumeration`

_Run it after uploading the config files or just delete the config commands(-config, -pc)._ 

In [ ]:
!./tools/amass_linux_amd64/amass enum -passive -d "$domain" -o "$domain".amass.txt -silent -config "config.ini" &> /dev/null

In [ ]:
!./tools/subfinder -d $domain -o "$domain".subfinder.txt -all -t 20 -nW -silent -max-time 120 -timeout 6  -pc "provider-config.yaml" &> /dev/null

`2. Merge lists, cleaning root domains`

In [ ]:
# file merge
inputlist = []
inputfiles = [domain+".amass.txt",domain+".subfinder.txt"]
for i in inputfiles:
    with open(i, "r", encoding="utf-8") as f:
            inputlist.extend(f.readlines())
print(len(inputlist))
# clean domains
itemlist = []
for i in inputlist:
    if "xn--" in i:
        continue
    item = i.split(f".{domain}")[0].split(".")[-1] + f".{domain}"
    if item not in itemlist:
        itemlist.append(item)

# itemlist.sort()
print("total domains: ", len(itemlist))
with open(domain+"_final.txt", "w", encoding="utf-8") as f:
    for i in itemlist:
        f.write("%s\n" % i)

`3. Validating the domain list with httpx`

In [ ]:
!./tools/httpx -l "$domain"_final.txt -t 100 -timeout 6 -json -o "$domain"_final.jsonl -silent &> /dev/null

`4. Generating json file and url list from httpx output`

In [ ]:
# !sed r 1.jsonl 2.jsonl > 3.jsonl

In [ ]:
import json
with open(domain+'_final.jsonl',encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

print("total domains: ", len(data))
out=json.dumps(data) #.encode("utf-8").decode('unicode-escape')

with open(domain+".json","w",encoding="utf-8") as f:
    f.write(out)

#json to urls.txt
with open(domain+".json","r",encoding="utf-8") as f:
  jdata=json.load(f)

urls=[]
for i in jdata:
  item=i["url"].split("//")[1].split(":")[0]
  if item not in urls:
    urls.append(item)
urls.sort()
with open(domain+".txt", "w", encoding="utf-8") as f:
    for i in urls:
        f.write("%s\n" % i)